In [1]:
import requests
import execjs
import pandas as pd


In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36 Edg/92.0.902.67',
}
leagues_url='http://info.310win.com/jsData/infoHeader.js'
leagues_data=requests.get(leagues_url,headers=headers)


In [3]:
if leagues_data.status_code==200:
    leagues_data_js=execjs.compile(str(leagues_data.text)[1:])
leagues_list=leagues_data_js.eval('arr')


In [4]:
leagues_list

[['InfoID_1',
  '英格兰',
  'images/20130121174918.jpg',
  '1',
  ['36,英超,1,0,2021-2022,2020-2021,2019-2020,2018-2019,2017-2018,2016-2017,2015-2016,2014-2015,2013-2014,2012-2013,2011-2012,2010-2011,2009-2010,2008-2009,2007-2008,2006-2007,2005-2006,2004-2005,2003-2004',
   '37,英冠,1,1,2021-2022,2020-2021,2019-2020,2018-2019,2017-2018,2016-2017,2015-2016,2014-2015,2013-2014,2012-2013,2011-2012,2010-2011,2009-2010,2008-2009,2007-2008,2006-2007,2005-2006,2004-2005',
   '39,英甲,1,1,2021-2022,2020-2021,2019-2020,2018-2019,2017-2018,2016-2017,2015-2016,2014-2015,2013-2014,2012-2013,2011-2012,2010-2011,2009-2010,2008-2009,2007-2008,2006-2007,2005-2006,2004-2005',
   '35,英乙,1,1,2021-2022,2020-2021,2019-2020,2018-2019,2017-2018,2016-2017,2015-2016,2014-2015,2013-2014,2012-2013,2011-2012,2010-2011,2009-2010,2008-2009,2007-2008,2006-2007,2005-2006,2004-2005',
   '146,英议联,1,1,2021-2022,2020-2021,2019-2020,2018-2019,2017-2018,2016-2017,2015-2016,2014-2015,2013-2014,2012-2013,2011-2012,2010-2011,2009-2010

In [ ]:
target=[]
for i in leagues_list:
    data={}
    data['country_name']=i[1]
    for t in i[4]:
        t=t.split(',')
        data['id']=t[0]
        data['lg_name']=t[1]
        data['seasons']=t[4+3:7+4]
        break
    target.append(data)

In [ ]:
for tournament in target[:1]:
    for year in tournament['seasons']:
        league_detail_url='http://info.310win.com/jsData/matchResult/'+year+'/s'+str(tournament['id'])+'.js'
        league_detail_data=requests.get(league_detail_url,headers=headers)
        if league_detail_data.status_code!=200:
            print('err')
        else:
            try:
                league_detail_data_js=execjs.compile('var jh= {};'+league_detail_data.text[1:])
                league_detail_name=league_detail_data_js.eval('arrLeague')[2]
                league_detail_team_row=league_detail_data_js.eval('arrTeam')
                league_detail_team={}
                for i in league_detail_team_row:
                    league_detail_team[i[0]]=i[1]
                league_detail_match=league_detail_data_js.eval('jh')
                all_match=[]
                for r in sorted(list(league_detail_match.keys())):
                    for m in league_detail_match[r]:
                        match={}
                        match['id']=m[0]
                        match['time']=m[3]
                        match['host']=league_detail_team[m[4]]
                        match['guest']=league_detail_team[m[5]]
                        match['full_res']=m[6]
                        match['half_res']=m[7]
                        match['give_full']=m[10]
                        match['give_half']=m[11]
                        match['sc_full']=m[12]
                        match['sc_half']=m[13]
                        all_match.append(match)

                        op_url='http://1x2d.win007.com/'+ str(match['id'])+'.js'
                        op_data=requests.get(op_url,headers=headers)
                        op_data_js=execjs.compile(op_data.text)
                        last_op=op_data_js.eval('gameDetail')
                        org_op=op_data_js.eval('game')
                        avg_org_op=[0,0,0]
                        for i in range(len(org_op)):
                            org_op[i] = org_op[i].split('|')
                            avg_org_op[0]+=float(org_op[i][3])
                            avg_org_op[1]+=float(org_op[i][4])
                            avg_org_op[2]+=float(org_op[i][5])
                        avg_org_op=[avg_org_op[0]/len(org_op),avg_org_op[1]/len(org_op),avg_org_op[2]/len(org_op)]
                        avg_last_op=[0,0,0]
                        for i in range(len(last_op)):
                            last_op[i] = last_op[i].split('|')
                            avg_last_op[0]+=float(last_op[i][0][last_op[i][0].index('^')+1:])
                            avg_last_op[1]+=float(last_op[i][1])
                            avg_last_op[2]+=float(last_op[i][2])
                        avg_last_op=[avg_last_op[0]/len(last_op),avg_last_op[1]/len(last_op),avg_last_op[2]/len(last_op)]
                        match['op_org_host']=avg_org_op[0]
                        match['op_org_equal']=avg_org_op[1]
                        match['op_org_guest']=avg_org_op[2]
                        match['op_last_host']=avg_last_op[0]
                        match['op_last_equal']=avg_last_op[1]
                        match['op_last_guest']=avg_last_op[2]
                        yp_url='http://www.310win.com/handicap/'+str(match['id'])+'.html'
                        yp_data=requests.get(yp_url,headers=headers)
                        if yp_data.status_code==200:
                            yp_tb=pd.read_html(yp_data.text)
                            yp_tb=yp_tb[1]
                            yp_tb.drop(index=[0,1])
                            del yp_tb[list(yp_tb.columns)[8]]
                            del yp_tb[list(yp_tb.columns)[7]]
                            dp=[]
                            for i in range(yp_tb.shape[0]):
                                if not '金宝博' in yp_tb.iloc[i][0]:dp.append(i)
                            yp_tb=yp_tb.drop(index=dp)
                            yp_tb=yp_tb.iloc[0].tolist()[1:]
                            match['yp_org_give_full']=yp_tb[1]
                            match['yp_last_give_half']=yp_tb[4]
                            match['yp_org_host']=yp_tb[0]
                            match['yp_org_guest']=yp_tb[2]
                            match['yp_last_host']=yp_tb[3]
                            match['yp_last_guest']=yp_tb[5]
                        else:
                            print(yp_data.status_code)
                        ball_url='http://www.310win.com/overunder/1904139.html'
                        ball_data=requests.get(ball_url,headers=headers)
                        ball_tb=pd.read_html(ball_data.text)[1]
                        ball_tb.drop(index=[0,1])
                        del ball_tb[list(ball_tb.columns)[8]]
                        del ball_tb[list(ball_tb.columns)[7]]
                        dp=[]
                        for i in range(ball_tb.shape[0]):
                            if not '金宝博' in ball_tb.iloc[i][0]:dp.append(i)
                        ball_tb=ball_tb.drop(index=dp)
                        ball_tb=ball_tb.iloc[0].tolist()[1:]0.87	2.5/3	1.03	1.00	2.5/3	0.92
                        match['org_ball']=ball_tb[1]
                        match['last_ball']=ball_tb[4]
                        match['org_big_ball']=ball_tb[0]
                        match['org_small_ball']=ball_tb[2]
                        match['last_big_ball']=ball_tb[3]
                        match['last_small_ball']=ball_tb[5]
                        break
                    break
            except:print(tournament['country_name']+tournament['lg_name']+year+'无法获取',s)
            break
print(pd.Series(match))
            

In [ ]:
ball_url='http://www.310win.com/overunder/1904139.html'
ball_data=requests.get(ball_url,headers=headers)
ball_tb=pd.read_html(ball_data.text)[1]
ball_tb.drop(index=[0,1])
del ball_tb[list(ball_tb.columns)[8]]
del ball_tb[list(ball_tb.columns)[7]]
dp=[]
for i in range(ball_tb.shape[0]):
    if not '金宝博' in ball_tb.iloc[i][0]:dp.append(i)
ball_tb=ball_tb.drop(index=dp)
ball_tb=ball_tb.iloc[0].tolist()[1:]

In [7]:
t=[{'id': 1551741, 'time': '2018-08-11 03:00', 'host': '曼彻斯特联', 'guest': '莱切斯特城', 'full_res': '2-1', 'half_res': '1-0', 'give_full': 1.25, 'give_half': 0.5, 'sc_full': '2.5/3', 'sc_half': '1', 'op_org_host': 1.4020673076923074, 'op_org_equal': 4.441730769230767, 'op_org_guest': 7.913269230769231, 'op_last_host': 1.5319711538461551, 'op_last_equal': 3.953221153846153, 'op_last_guest': 7.097163461538463, 'yp_org_give_full': '球半', 'yp_last_give_half': '一球', 'yp_org_host': '1.01', 'yp_org_guest': '0.89', 'yp_last_host': '0.98', 'yp_last_guest': '0.96', 'org_ball': '2.5/3', 'last_ball': '2.5/3', 'org_big_ball': '0.87', 'org_small_ball': '1.03', 'last_big_ball': '1.00', 'last_small_ball': '0.92'}, {'id': 1394661, 'time': '2017-08-12 02:45', 'host': '阿森纳', 'guest': '莱切斯特城', 'full_res': '4-3', 'half_res': '2-2', 'give_full': 1.25, 'give_half': 0.5, 'sc_full': '3', 'sc_half': '1/1.5', 'op_org_host': 1.3867701863354032, 'op_org_equal': 4.720527950310559, 'op_org_guest': 7.579720496894415, 'op_last_host': 1.488043478260869, 'op_last_equal': 4.450186335403724, 'op_last_guest': 6.491894409937882, 'yp_org_give_full': '一球/球半', 'yp_last_give_half': '一球', 'yp_org_host': '0.91', 'yp_org_guest': '0.99', 'yp_last_host': '0.82', 'yp_last_guest': '1.13', 'org_ball': '2.5/3', 'last_ball': '2.5/3', 'org_big_ball': '0.87', 'org_small_ball': '1.03', 'last_big_ball': '1.00', 'last_small_ball': '0.92'}, {'id': 1247648, 'time': '2016-08-13 19:30', 'host': '赫尔城', 'guest': '莱切斯特城', 'full_res': '2-1', 'half_res': '1-0', 'give_full': -0.25, 'give_half': 0, 'sc_full': '2/2.5', 'sc_half': '0.5/1', 'op_org_host': 3.4086799999999995, 'op_org_equal': 3.213800000000004, 'op_org_guest': 2.1775999999999978, 'op_last_host': 4.309600000000002, 'op_last_equal': 3.405879999999998, 'op_last_guest': 1.9101600000000005, 'yp_org_give_full': '受平手/半球', 'yp_last_give_half': '受半球', 'yp_org_host': '0.91', 'yp_org_guest': '0.99', 'yp_last_host': '1.03', 'yp_last_guest': '0.91', 'org_ball': '2.5/3', 'last_ball': '2.5/3', 'org_big_ball': '0.87', 'org_small_ball': '1.03', 'last_big_ball': '1.00', 'last_small_ball': '0.92'}, {'id': 1130197, 'time': '2015-08-08 19:45', 'host': '曼彻斯特联', 'guest': '托特纳姆热刺', 'full_res': '1-0', 'half_res': '1-0', 'give_full': 0.75, 'give_half': 0.25, 'sc_full': '2.5', 'sc_half': '1', 'op_org_host': 1.6390251572327035, 'op_org_equal': 3.7608490566037744, 'op_org_guest': 5.168427672955973, 'op_last_host': 1.6251572327044028, 'op_last_equal': 3.8418238993710636, 'op_last_guest': 5.487861635220124, 'yp_org_give_full': '半球/一球', 'yp_last_give_half': '一球', 'yp_org_host': '1.01', 'yp_org_guest': '0.89', 'yp_last_host': '1.09', 'yp_last_guest': '0.86', 'org_ball': '2.5/3', 'last_ball': '2.5/3', 'org_big_ball': '0.87', 'org_small_ball': '1.03', 'last_big_ball': '1.00', 'last_small_ball': '0.92'}]


,id,time,host,guest,full_res,half_res,give_full,give_half,sc_full,sc_half,...,yp_org_host,yp_org_guest,yp_last_host,yp_last_guest,org_ball,last_ball,org_big_ball,org_small_ball,last_big_ball,last_small_ball
0,1551741,2018-08-11 03:00,曼彻斯特联,莱切斯特城,2-1,1-0,1.25,0.50,2.5/3,1,...,1.01,0.89,0.98,0.96,2.5/3,2.5/3,0.87,1.03,1.00,0.92
1,1394661,2017-08-12 02:45,阿森纳,莱切斯特城,4-3,2-2,1.25,0.50,3,1/1.5,...,0.91,0.99,0.82,1.13,2.5/3,2.5/3,0.87,1.03,1.00,0.92
2,1247648,2016-08-13 19:30,赫尔城,莱切斯特城,2-1,1-0,-0.25,0.00,2/2.5,0.5/1,...,0.91,0.99,1.03,0.91,2.5/3,2.5/3,0.87,1.03,1.00,0.92
3,1130197,2015-08-08 19:45,曼彻斯特联,托特纳姆热刺,1-0,1-0,0.75,0.25,2.5,1,...,1.01,0.89,1.09,0.86,2.5/3,2.5/3,0.87,1.03,1.00,0.92


In [1]:
import requests
import execjs
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36 Edg/92.0.902.67',
}
headers={
'Referer':'http://info.310win.com/cn/League/2020-2021/36.html',
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36 Edg/92.0.902.67'}
import requests
import execjs
import pandas as pd
import datetime
import time

leagues_url='http://info.310win.com/jsData/infoHeader.js'
leagues_data=requests.get(leagues_url,headers=headers)
if leagues_data.status_code==200:
    leagues_data_js=execjs.compile(str(leagues_data.text)[1:])
leagues_list=leagues_data_js.eval('arr')
target=[]
for i in leagues_list:
    data={}
    data['country_name']=i[1]
    for t in i[4]:
        t=t.split(',')
        data['id']=t[0]
        data['lg_name']=t[1]
        data['seasons']=t[4+3:7+4]
        break
    target.append(data)

for tournament in target[:1]:
    for year in tournament['seasons']:
        league_detail_url='http://info.310win.com/jsData/matchResult/'+year+'/s'+str(tournament['id'])+'.js'
        league_detail_data=requests.get(league_detail_url,headers=headers)
        if league_detail_data.status_code!=200:
            print('err')
        else:
            print('爬取',tournament['country_name'],year,tournament['lg_name'],'中',datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            league_detail_data_js=execjs.compile('var jh= {};'+league_detail_data.text[1:])
            league_detail_name=league_detail_data_js.eval('arrLeague')[2]
            league_detail_team_row=league_detail_data_js.eval('arrTeam')
            league_detail_team={}
            for i in league_detail_team_row:
                league_detail_team[i[0]]=i[1]
            league_detail_match=league_detail_data_js.eval('jh')
            all_match=[]
            for r in sorted(list(league_detail_match.keys())):
                for m in league_detail_match[r]:
                    match={}
                    match['id']=m[0]
                    match['time']=m[3]
                    match['host']=league_detail_team[m[4]]
                    match['guest']=league_detail_team[m[5]]
                    match['full_res']=m[6]
                    match['half_res']=m[7]
                    match['give_full']=m[10]
                    match['give_half']=m[11]
                    match['sc_full']=m[12]
                    match['sc_half']=m[13]
                    op_url='http://1x2d.win007.com/'+ str(match['id'])+'.js'
                    op_data=requests.get(op_url,headers=headers)
                    op_data_js=execjs.compile(op_data.text)
                    last_op=op_data_js.eval('gameDetail')
                    org_op=op_data_js.eval('game')
                    avg_org_op=[0,0,0]
                    for i in range(len(org_op)):
                        org_op[i] = org_op[i].split('|')
                        avg_org_op[0]+=float(org_op[i][3])
                        avg_org_op[1]+=float(org_op[i][4])
                        avg_org_op[2]+=float(org_op[i][5])
                    avg_org_op=[avg_org_op[0]/len(org_op),avg_org_op[1]/len(org_op),avg_org_op[2]/len(org_op)]
                    avg_last_op=[0,0,0]
                    for i in range(len(last_op)):
                        last_op[i] = last_op[i].split('|')
                        avg_last_op[0]+=float(last_op[i][0][last_op[i][0].index('^')+1:])
                        avg_last_op[1]+=float(last_op[i][1])
                        avg_last_op[2]+=float(last_op[i][2])
                    avg_last_op=[avg_last_op[0]/len(last_op),avg_last_op[1]/len(last_op),avg_last_op[2]/len(last_op)]
                    match['op_org_host']=avg_org_op[0]
                    match['op_org_equal']=avg_org_op[1]
                    match['op_org_guest']=avg_org_op[2]
                    match['op_last_host']=avg_last_op[0]
                    match['op_last_equal']=avg_last_op[1]
                    match['op_last_guest']=avg_last_op[2]
                    #time.sleep(3)
                    yp_url='http://www.310win.com/handicap/'+str(match['id'])+'.html'
                    yp_data=requests.get(yp_url,headers=headers)
                    #time.sleep(3)
                    yp_tb=pd.read_html(yp_data.text)
                    yp_tb=yp_tb[1]
                    yp_tb.drop(index=[0,1])
                    del yp_tb[list(yp_tb.columns)[8]]
                    del yp_tb[list(yp_tb.columns)[7]]
                    dp=[]
                    for i in range(yp_tb.shape[0]):
                        if not '金宝博' in yp_tb.iloc[i][0]:dp.append(i)
                    yp_tb=yp_tb.drop(index=dp)
                    yp_tb=yp_tb.iloc[0].tolist()[1:]
                    match['yp_org_give_full']=yp_tb[1]
                    match['yp_last_give_half']=yp_tb[4]
                    match['yp_org_host']=yp_tb[0]
                    match['yp_org_guest']=yp_tb[2]
                    match['yp_last_host']=yp_tb[3]
                    match['yp_last_guest']=yp_tb[5]
                    #time.sleep(3)
                    ball_url='http://www.310win.com/overunder/'+str(match['id'])+'.html'
                    ball_data=requests.get(ball_url,headers=headers)
                    #time.sleep(3)
                    ball_tb=pd.read_html(ball_data.text)[1]
                    ball_tb.drop(index=[0,1])
                    del ball_tb[list(ball_tb.columns)[8]]
                    del ball_tb[list(ball_tb.columns)[7]]
                    dp=[]
                    for i in range(ball_tb.shape[0]):
                        if not '金宝博' in ball_tb.iloc[i][0]:dp.append(i)
                    ball_tb=ball_tb.drop(index=dp)
                    ball_tb=ball_tb.iloc[0].tolist()[1:]
                    match['org_ball']=ball_tb[1]
                    match['last_ball']=ball_tb[4]
                    match['org_big_ball']=ball_tb[0]
                    match['org_small_ball']=ball_tb[2]
                    match['last_big_ball']=ball_tb[3]
                    match['last_small_ball']=ball_tb[5]
                    all_match.append(match)
                dt=pd.DataFrame(all_match)
                dt.to_excel(tournament['country_name']+tournament['lg_name']+year+'.xlsx')

爬取 英格兰 2018-2019 英超 中 2021-08-16 00:11:36


In [5]:
all_data.columns

NameError: name 'all_data' is not defined